In [11]:
import os
import numpy as np
import cv2
import time
from imageio import imwrite

In [ ]:
# in-notebook display
from IPython.display import clear_output
import PIL.Image
from io import BytesIO
import IPython.display

### define paths

In [2]:
data_paths = r"C:\Users\alexa\Documents\data\mlproj_data"
vid_file = r"Test video for Object Detection __ TRIDE.mp4"
vid_path = os.path.join(data_paths,vid_file)

### define display parameters

In [4]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = BytesIO()
    
    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)
    
    return IPython.display.Image(data=f.getvalue())

### define LK parameters

In [5]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

In [14]:
cap = cv2.VideoCapture(vid_path)

In [15]:
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

In [16]:
total_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
dur = total_frame_count / fps
save_frame_freq = int(dur)

In [17]:
dipy = IPython.display.display("", display_id=1)
count_frames = 0
max_frame_count = 100
img_count = 1
reset_mask = True
t0 = time.time()

while True:
    try:
        count_frames += 1
        ret, frame = cap.read()
       
        # check stream over
        if not ret:
            tend = time.time()
            print()
            cap.release()
            IPython.display.clear_output()
            print ("Stream ended")
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
        img = cv2.add(frame,mask)
        
        # display
        im = array_to_image(img)
        #dipy.update(im)
        
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)
        
        # save mask
        if count_frames %  save_frame_freq == 0:
            mask_amend = mask.copy()
            mask_amend[mask_amend == 0] = 255
            imwrite('mask_{}.jpg'.format(img_count), mask_amend)
            img_count += 1
            if reset_mask:
                mask = np.zeros_like(old_frame)
        
        if count_frames > max_frame_count:
            tend = time.time()
            print()
            cap.release()
            IPython.display.clear_output()
            print ("reached max frame count")
            break
            
    except KeyboardInterrupt:
        tend = time.time()
        print()
        cap.release()
        IPython.display.clear_output()
        print ("Stream stopped")
        break
tend = time.time()
ttot = tend - t0

reached max frame count
